# Mobile App for Lottery Addiction

Many people start playing the lottery for fun, but for some this activity turns into a habit which eventually escalates into addiction. Like other compulsive gamblers, lottery addicts soon begin spending from their savings and loans, they start to accumulate debts, and eventually engage in desperate behaviors like theft.

A medical institute that aims to prevent and treat gambling addictions wants to build a dedicated mobile app to help lottery addicts better estimate their chances of winning. The institute has a team of engineers that will build the app, but they need us to create the logical core of the app and calculate probabilities.

For the first version of this app we are going to focus on the [6/49](https://en.wikipedia.org/wiki/Lotto_6/49) lottery and build functions that enables users to answers questions like:
- What is the probability of winning the big prize with a single ticket?
- What is the probability of winning the big prize if we play 40 different tickets (or any other number)?
- What is the probability of having at least five (or four, or three, or two) winning numbers on a single ticket?

In [232]:
# function that calculates factorials
def factorial(n):
    final_product = 1
    for i in range(n, 0, -1):
        final_product *= i
    return final_product

# function that calculates combinations
def combinations(n, k):
    return factorial(n) / (factorial(k) * factorial(n-k))

In the 6/49 lottery, six numbers are drawn from a set of 49 numbers that range from 1 to 49. A player wins the big prize if the six numbers on their tickets match all the six numbers drawn. If a player has a ticket with the numbers {13, 22, 24, 27, 42, 44}, he only wins the big prize if the numbers drawn are {13, 22, 24, 27, 42, 44}. If only one number differs, he doesn't win.

For the first version of the app, we want players to be able to calculate the probability of winning the big prize with the various numbers they play on a single ticket (for each ticket a player chooses six numbers out of 49). So, we'll start by building a function that calculates the probability of winning the big prize for any given ticket.

We discussed with the engineering team of the medical institute, and they told us we need to be aware of the following details when we write the function:

- Inside the app, the user inputs six different numbers from 1 to 49.
- Under the hood, the six numbers will come as a Python list, which will serve as the single input to our function.
- The engineering team wants the function to print the probability value in a friendly way — in a way that people without any probability training are able to understand.

I'll now write the function.

In [233]:
def one_ticket_probability(list_six):
    possible_outcomes = combinations(49, 6)
    probability = 1 / possible_outcomes
    return "The probability that the numbers {} win the big prize are {}".format(list_six, probability)

In [234]:
# example of the output
one_ticket_probability([12, 14, 26, 34, 47, 48])

'The probability that the numbers [12, 14, 26, 34, 47, 48] win the big prize are 7.151123842018516e-08'

In the function above I calculated the probability of a person winning the big prize using the siz numbers of their choosing. The function returns the probability written out for any person to easily read. 

## Adding in historical data

For the first version of the app, however, users should also be able to compare their ticket against the historical lottery data in Canada and determine whether they would have ever won by now.

I'll now focus on exploring the historical data coming from the Canada 6/49 lottery. The data set can be downloaded from [Kaggle](https://www.kaggle.com/datascienceai/lottery-dataset).

In [235]:
# read in the csv
import pandas as pd

hist_data = pd.read_csv(r'C:\Users\chizj\Downloads\DataQuest CSVs\649.csv')

hist_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3665 entries, 0 to 3664
Data columns (total 11 columns):
PRODUCT            3665 non-null int64
DRAW NUMBER        3665 non-null int64
SEQUENCE NUMBER    3665 non-null int64
DRAW DATE          3665 non-null object
NUMBER DRAWN 1     3665 non-null int64
NUMBER DRAWN 2     3665 non-null int64
NUMBER DRAWN 3     3665 non-null int64
NUMBER DRAWN 4     3665 non-null int64
NUMBER DRAWN 5     3665 non-null int64
NUMBER DRAWN 6     3665 non-null int64
BONUS NUMBER       3665 non-null int64
dtypes: int64(10), object(1)
memory usage: 315.1+ KB


In [236]:
# first three rows
hist_data.head(3)

,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
0,649,1,0,6/12/1982,3,11,12,14,41,43,13
1,649,2,0,6/19/1982,8,33,36,37,39,41,9
2,649,3,0,6/26/1982,1,6,23,24,27,39,34


In [237]:
# last three rows
hist_data.tail(3)

,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
3662,649,3589,0,6/13/2018,6,22,24,31,32,34,16
3663,649,3590,0,6/16/2018,2,15,21,31,38,49,8
3664,649,3591,0,6/20/2018,14,24,31,35,37,48,17


I'm now going to write a function that will enable users to compare their ticket against the historical lottery data in Canada and determine whether they would have ever won by now.

The engineering team told us that we need to be aware of the following details:

- Inside the app, the user inputs six different numbers from 1 to 49.
- Under the hood, the six numbers will come as a Python list and serve as an input to our function.
- The engineering team wants us to write a function that prints:
    - the number of times the combination selected occurred in the Canada data set; and
    - the probability of winning the big prize in the next drawing with that combination.

In [238]:
# this function will return all the winning numbers from hist_data as a ptython set

def extract_numbers(row):
    row = row[4:10]
    row = set(row.values)
    return row

# apply the function to the data set
winning_numbers = hist_data.apply(extract_numbers, axis=1)

# check the first 5 numbers
winning_numbers.head()

0    {3, 41, 11, 12, 43, 14}
1    {33, 36, 37, 39, 8, 41}
2     {1, 6, 39, 23, 24, 27}
3     {3, 9, 10, 43, 13, 20}
4    {34, 5, 14, 47, 21, 31}
dtype: object

In [239]:
# this function will tell the user how many times their set of numbers matched a set of winning numbers from hist_data

def check_historical_occurance(user_numbers, winning_numbers):
    user_numbers = set(user_numbers)
    number_of_matches = 0
    
    for row in winning_numbers:
        if user_numbers == row:
            number_of_matches += 1
    
    return print('The number of times this combination of numbers has won the big prize is {}.'.format(number_of_matches))
        

I'll now check the functions to see if they work as expected.

In [240]:
# this should return 0 matches
check_historical_occurance([5,17,44,32,27,31], winning_numbers)

The number of times this combination of numbers has won the big prize is 0.


In [241]:
# this should return 1 match
check_historical_occurance([33,37,36,8,39,41], winning_numbers)

The number of times this combination of numbers has won the big prize is 1.


## Chances of winning for any number of tickets

So far, I've wrote two functions:

- one_ticket_probability() — calculates the probability of winning the big prize with a single ticket
- check_historical_occurrence() — checks whether a certain combination has occurred in the Canada lottery data set

Lottery addicts usually play more than one ticket on a single drawing, thinking that this might increase their chances of winning significantly. The purpose of this app is to help them better estimate their chances of winning — next, I'll write a function that will allow the users to calculate the chances of winning for any number of different tickets.

The engineering team gave me the following information:

- The user will input the number of different tickets they want to play (without inputting the specific combinations they intend to play).
- Our function will see an integer between 1 and 13,983,816 (the maximum number of different tickets).
- The function should print information about the probability of winning the big prize depending on the number of different tickets played.

I'll now create the function.

In [242]:
def multi_ticket_probability(number_of_tickets):
    total_outcomes = combinations(49,6)
    successful_outcomes = number_of_tickets
    probability = (successful_outcomes / total_outcomes) * 100
    
    return print('You will have a {:.6f}% chance to win the big prize when you play {} tickets.'.format(probability, number_of_tickets))

In [243]:
# testing the function
for test_tickets in [1, 10, 100, 10000, 1000000, 6991908, 13983816]:
    multi_ticket_probability(test_tickets)
    print('--------------------------')

You will have a 0.000007% chance to win the big prize when you play 1 tickets.
--------------------------
You will have a 0.000072% chance to win the big prize when you play 10 tickets.
--------------------------
You will have a 0.000715% chance to win the big prize when you play 100 tickets.
--------------------------
You will have a 0.071511% chance to win the big prize when you play 10000 tickets.
--------------------------
You will have a 7.151124% chance to win the big prize when you play 1000000 tickets.
--------------------------
You will have a 50.000000% chance to win the big prize when you play 6991908 tickets.
--------------------------
You will have a 100.000000% chance to win the big prize when you play 13983816 tickets.
--------------------------


## Multiple winning tickets

I'm now going to write one more function to allow the users to calculate probabilities for two, three, four, or five winning numbers.

For extra context, in most 6/49 lotteries there are smaller prizes if a player's ticket match two, three, four, or five of the six numbers drawn. As a consequence, the users might be interested in knowing the probability of having two, three, four, or five winning numbers.

These are the engineering details I'll need to be aware of:

- Inside the app, the user inputs:
    - six different numbers from 1 to 49; and
    - an integer between 2 and 5 that represents the number of winning numbers expected
- Our function prints information about the probability of having the inputted number of winning numbers.

I'll now create the function.

In [244]:
def probability_6_less(number):
    n_combinations_tickets = combinations(6,number)
    n_combinations_remaining = combinations(43, 6-number)
    n_successful_outcomes = n_combinations_tickets * n_combinations_remaining
    
    total_possible_outcomes = combinations(49, 6)
    probability = (n_successful_outcomes / total_possible_outcomes) * 100
    
    return print('The percentage chance of winning the lottery with your chosen numbers is {:.6f}%.'.format(probability))

In [245]:
# testing the function
for test_tickets in [2,3,4,5]:
    probability_6_less(test_tickets)
    print('-------------------------')

The percentage chance of winning the lottery with your chosen numbers is 13.237803%.
-------------------------
The percentage chance of winning the lottery with your chosen numbers is 1.765040%.
-------------------------
The percentage chance of winning the lottery with your chosen numbers is 0.096862%.
-------------------------
The percentage chance of winning the lottery with your chosen numbers is 0.001845%.
-------------------------
